In [ ]:
from redmine import Redmine

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import csv

In [ ]:
redmine = Redmine(
    'https://www.scm.verwaltung.uni-muenchen.de/internetdienste/',
    #'https://localhost/internetdienste/',
    #username='admin',
    #password='admin',
    key='',
    requests={'verify': False})

In [ ]:
login_data = set()
with open('userLogins-2016-03-07.csv', 'r', newline='') as csvfile:
    login_reader = csv.DictReader(csvfile, dialect='excel')
    for row in login_reader:
        login_data.add(row['LMU-Benutzerkennung'].lower().strip())


In [ ]:
group_data = set()
with open('dumpFionaGruppenUsers_20160407.csv', 'r', newline='') as csvfile:
    group_reader = csv.DictReader(csvfile, delimiter=';')
    for row in group_reader:
        for name in row['Mitglieder'].lower().strip().split(','):
            group_data.add(name)

In [ ]:
group_data.remove('')

In [ ]:
not_logged_in = group_data - login_data

In [ ]:
len(not_logged_in)

In [ ]:
_all_contacts = redmine.contact.all()

In [ ]:
contacts = {}
for contact in _all_contacts:
    cp = contact.custom_fields.get(7).value
    if cp:
        cp = cp.lower()
        contacts[cp] = contact

In [ ]:
len(contacts)

In [ ]:
for cp in not_logged_in:
    if cp in contacts.keys():
        taglist = contacts[cp].tag_list
        taglist.append('UserCheck')
        redmine.contact.update(contacts[cp].id, tag_list=taglist)

In [ ]:
new_contacts = group_data - set(contacts.keys())

In [ ]:
len(new_contacts)

In [ ]:
new_contacts